# Some example SDFITSIO operations with masking


In [1]:
import numpy as np
import numpy.ma as ma


import sdfitsio as sdf
from importlib import reload
reload(sdf);

Create a masked array 'a' with a typical LMT structure

In [2]:
ntime = 1000
nbeam = 16
npol = 2
nband = 6
nchan = 256

adim=(ntime,nbeam,npol,nband,nchan)
%time a = sdf.gen_data(adim)

print('Size of data',sdf.dimsize(adim))

CPU times: user 929 ms, sys: 26.9 ms, total: 956 ms
Wall time: 955 ms
Size of data 49152000


Write it as SDFITS

In [3]:
%time sdf.my_write_sdfits('junk.fits',a)

SHAPE= (1000, 16, 2, 6, 256)
NAXIS2= 192000
NCHAN= 256
(192000,)
(192000, 256)
Written junk.fits
CPU times: user 351 ms, sys: 1.33 s, total: 1.68 s
Wall time: 1.69 s


Set up a masking operation. Low level method. Any dimensions skipped, imply all elements are selected, i.e. [:]

In [4]:
mask = {}
mask[0] = '0:110'     # time 
#mask[1] = '3'        # beam
#mask[2] = '0'        # pol
#mask[3] = '4'        # band
mask[4] = '81:83'    # chan


In [5]:
%time sdf.data_mask(a, mask)


DATA_MASK:  {0: '0:110', 4: '81:83'}
CMD: data.mask[0:110,:,:,:,81:83] = True
CPU times: user 1.37 ms, sys: 0 ns, total: 1.37 ms
Wall time: 1.02 ms


Report what fraction was masked

In [6]:
%time sdf.data_masked(a)

data_masked: 42240 / 49152000 = 0.0859375%
CPU times: user 52.7 ms, sys: 0 ns, total: 52.7 ms
Wall time: 51.8 ms


In [7]:
%time sdf.my_write_sdfits('junk_masked.fits',a)

SHAPE= (1000, 16, 2, 6, 256)
NAXIS2= 192000
NCHAN= 256
(192000,)
(192000, 256)
Written junk_masked.fits
CPU times: user 237 ms, sys: 1.34 s, total: 1.58 s
Wall time: 1.58 s


Read back the original unmasked

In [8]:
%time b=sdf.my_read('junk.fits')

File:      junk.fits
Size:      4 cols x 192000 rows
Telescope: LMT/GTM
Object:    ['NOISE']
DateObs:   0.0
DIMS2: (1000, 16, 2, 6, 256)
(192000, 256)
(1000, 16, 2, 6, 256)
Data.sum() =  -4922999.5
CPU times: user 102 ms, sys: 9.93 ms, total: 112 ms
Wall time: 111 ms


In [9]:
b.shape

(1000, 16, 2, 6, 256)

In [10]:
%time sdf.data_masked(b)

data_masked: 0 / 49152000 = 0%
CPU times: user 41.8 ms, sys: 10.5 ms, total: 52.3 ms
Wall time: 52.7 ms


In [11]:
%time c = sdf.my_read('junk_masked.fits')

File:      junk_masked.fits
Size:      4 cols x 192000 rows
Telescope: LMT/GTM
Object:    ['NOISE']
DateObs:   0.0
DIMS2: (1000, 16, 2, 6, 256)
(192000, 256)
(1000, 16, 2, 6, 256)
Data.sum() =  nan
CPU times: user 108 ms, sys: 3.16 ms, total: 111 ms
Wall time: 111 ms


In [12]:
%time sdf.data_masked(c)

data_masked: 42240 / 49152000 = 0.0859375%
CPU times: user 44 ms, sys: 7.99 ms, total: 52 ms
Wall time: 52.3 ms


Indeed, the masking have now been applied (as NaN) in the SDFITS file. masks cannot be undone here, since we did not save the masks. Thus users are encouraged to save them seperately.